In [1]:
import pandas as pd
import numpy as np
import gc
import os
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.neighbors import NearestNeighbors

In [2]:
%matplotlib notebook

In [3]:
BASE_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data"
# PAIR_DIST_THRESH = 0.25  # in meters
PAIR_DIST_THRESH = 2  # in meters
PAIR_ANGL_THRESH = 15.0  # in degrees


STATIC_FOLDER = "static"
DYNAMIC_FOLDER = "dynamic"
PCD_FOLDER = "_out"
PAIR_FILE = "pair_with_pose.csv"

GROUNDTRUTH_FILE = "groundTruth.csv"
PCD_IDX_COLUMN = 'step'

ANNOT_FILE = "annotation.csv"
# START_ANNOT_COLUMN = 'start'
# FINISH_ANNOT_COLUMN = 'end'



STATIC_PATH = os.path.join(BASE_PATH, STATIC_FOLDER)
DYNAMIC_PATH = os.path.join(BASE_PATH, DYNAMIC_FOLDER)

In [4]:
def get_file_int(name):
    return int(name.split('.')[0])

def get_folder_int(name):
    return int(name)

In [5]:
def get_all_gt_df(base_folder, use_annot=False):
    
    pcd_list = []
    for sub_folder in sorted(os.listdir(base_folder), key=get_folder_int):
        # WARNING : Remove this if condition after use
#         if int(sub_folder) > 6:
#             continue
        pcd_path = os.path.join(base_folder, sub_folder, PCD_FOLDER)

        gt_file_path = os.path.join(base_folder, sub_folder, GROUNDTRUTH_FILE)
        if os.path.exists(gt_file_path):
            df_gt = pd.read_csv(gt_file_path)
        else:
            print("Error: Could not find ground truth csv at path : {}".format(gt_file_path))
            continue
        df_gt_pcd = df_gt.set_index(PCD_IDX_COLUMN)
        
        if use_annot:
            annot_file_path = os.path.join(base_folder, sub_folder, ANNOT_FILE)
            if os.path.exists(annot_file_path):
                df_annot = pd.read_csv(annot_file_path)
                feasible_pcd_idx_arr = df_annot[df_annot['annotation']]['step'].values
            else:
                print("Error: Could not find annotation csv at path : {}".format(annot_file_path))
                continue
#             feasible_start_list = df_annot[START_ANNOT_COLUMN].values
#             feasible_end_list   = df_annot[FINISH_ANNOT_COLUMN].values
#             feasible_pcd_idx_arr = np.concatenate([np.arange(start,end+1)\
#                                                    for start, end in zip(feasible_start_list, feasible_end_list)])

        for pcd_file in sorted(os.listdir(pcd_path), key=get_file_int):
            pcd_idx = int(pcd_file[:-4])
            if use_annot:
                if pcd_idx not in feasible_pcd_idx_arr:
                    continue
                    
            try:
                pcd_dict = df_gt_pcd.loc[pcd_idx]
                gt_dict = {}
                gt_dict['x'] = pcd_dict['location_x']
                gt_dict['y'] = pcd_dict['location_y']
                gt_dict['yaw'] = pcd_dict['rotation_yaw']
                gt_dict['path'] = os.path.join(pcd_path, pcd_file)
                pcd_list.append(gt_dict)
            except Exception as e:
                # print("Warning: Could not find groundtruth for {} . Ignoring it!".format(os.path.join(pcd_path, pcd_file)))
                pass
    return pd.DataFrame(pcd_list)

def get_relative_yaw(theta1, theta2):
    rel_yaw = theta2 - theta1
    if rel_yaw > 180:
        actual_rel_yaw = rel_yaw - 360
    elif rel_yaw <= -180:
        actual_rel_yaw = rel_yaw + 360
    else:
        actual_rel_yaw = rel_yaw
    return actual_rel_yaw

In [6]:
df_static = get_all_gt_df(STATIC_PATH)
df_dynamic = get_all_gt_df(DYNAMIC_PATH, use_annot=True)
# df_dynamic = get_all_gt_df(DYNAMIC_PATH)
gc.collect()

0

In [7]:
df_static.shape, df_dynamic.shape

((5298, 4), (13797, 4))

In [8]:
df_static.head()

,x,y,yaw,path
0,237.699997,129.75,179.999756,/home/saby/Projects/ati/data/data/datasets/Car...
1,237.699997,129.75,179.999756,/home/saby/Projects/ati/data/data/datasets/Car...
2,237.699997,129.75,179.999756,/home/saby/Projects/ati/data/data/datasets/Car...
3,237.699997,129.75,179.999756,/home/saby/Projects/ati/data/data/datasets/Car...
4,237.699997,129.75,179.999756,/home/saby/Projects/ati/data/data/datasets/Car...


In [9]:
static_gt_arr  = np.array([np.array([row['x'], row['y']]) for idx, row in df_static.iterrows()])
dynamic_gt_arr = np.array([np.array([row['x'], row['y']]) for idx, row in df_dynamic.iterrows()])

In [10]:
# Get all static points at a threshold radius from each dynamic point
nn_static = NearestNeighbors(radius=PAIR_DIST_THRESH)
nn_static.fit(static_gt_arr)
feasible_distance_list, feasible_distance_idx_list = \
            nn_static.radius_neighbors(dynamic_gt_arr, radius=PAIR_DIST_THRESH)

gc.collect()

24

In [11]:
# For all static idxs found for every dynamic point, see if we are in given orientation threshold
feasible_static_list = []

for dynamic_idx in tqdm_notebook(range(df_dynamic.shape[0])):
    feasible_flag = False
    
    feasible_ang_list = []
    feasible_static_idx_list = []
    more_feasible_distance_list = []
    
    
    if len(feasible_distance_idx_list[dynamic_idx]) > 0:
        dynamic_yaw = df_dynamic.iloc[dynamic_idx]['yaw']
        for static_distance, static_idx in zip(feasible_distance_list[dynamic_idx], feasible_distance_idx_list[dynamic_idx]):
            static_yaw = df_static.iloc[static_idx]['yaw']
            rel_yaw = np.abs(get_relative_yaw(static_yaw, dynamic_yaw)) # we get value between 0 and 180
            if rel_yaw < PAIR_ANGL_THRESH:
                feasible_flag = True
                feasible_ang_list.append(rel_yaw)
                feasible_static_idx_list.append(static_idx)
                more_feasible_distance_list.append(static_distance)
                
    feasible_static_dict = {}
    feasible_static_dict['flag'] = feasible_flag
    feasible_static_dict['dynamic_idx'] = dynamic_idx
    feasible_static_dict['static_idx_list'] = feasible_static_idx_list
    feasible_static_dict['distance_list'] = more_feasible_distance_list
    feasible_static_dict['angle_list'] = feasible_ang_list
    feasible_static_list.append(feasible_static_dict)
    
    
gc.collect()

/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


3

In [12]:
df_feasible = pd.DataFrame(feasible_static_list)
df_feasible.shape

(13797, 5)

In [13]:
# df_feasible[df_feasible['flag']]

In [14]:
corr_static_idx_list = []
corr_distance_list = []
corr_angle_list = []
for idx, row in df_feasible.iterrows():
    if row['flag']:
        corr_distance_list.append(np.min(row['distance_list']))
        
        which_idx = np.argmin(row['distance_list'])
        
        corr_static_idx = row['static_idx_list'][which_idx]
        corr_static_idx_list.append(corr_static_idx)
        
        corr_angle_idx = row['angle_list'][which_idx]
        corr_angle_list.append(corr_angle_idx)
    else:
        corr_distance_list.append(np.nan)
        corr_static_idx_list.append(-1)
        corr_angle_list.append(np.nan)
        
gc.collect()

0

In [15]:
df_feasible['static_idx'] = corr_static_idx_list
df_feasible['distance'] = corr_distance_list
df_feasible['angle'] = corr_angle_list

In [16]:
df_pair = df_feasible[df_feasible['flag']][['dynamic_idx', 'static_idx', 'distance', 'angle']].reset_index().drop(columns=['index'])
df_pair.shape

(9951, 4)

In [17]:
del df_feasible
gc.collect()

0

In [18]:
df_pair.head()

,dynamic_idx,static_idx,distance,angle
0,0,3813,0.315429,0.013123
1,1,3813,0.315429,0.013123
2,2,3813,0.315429,0.013123
3,3,3813,0.315429,0.013123
4,4,3813,0.315429,0.013123


In [19]:
df_pair['static_x'] = [df_static.iloc[int(row['static_idx'])]['x'] for idx, row in df_pair.iterrows()]
df_pair['static_y'] = [df_static.iloc[int(row['static_idx'])]['y'] for idx, row in df_pair.iterrows()]
df_pair['static_yaw'] = [df_static.iloc[int(row['static_idx'])]['yaw'] for idx, row in df_pair.iterrows()]

df_pair['dynamic_x'] = [df_dynamic.iloc[int(row['dynamic_idx'])]['x'] for idx, row in df_pair.iterrows()]
df_pair['dynamic_y'] = [df_dynamic.iloc[int(row['dynamic_idx'])]['y'] for idx, row in df_pair.iterrows()]
df_pair['dynamic_yaw'] = [df_dynamic.iloc[int(row['dynamic_idx'])]['yaw'] for idx, row in df_pair.iterrows()]

In [20]:
df_pair.head()

,dynamic_idx,static_idx,distance,angle,static_x,static_y,static_yaw,dynamic_x,dynamic_y,dynamic_yaw
0,0,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756
1,1,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756
2,2,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756
3,3,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756
4,4,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756


In [21]:
df_pair['pose_x'] = df_pair['dynamic_x'] - df_pair['static_x']
df_pair['pose_y'] = df_pair['dynamic_y'] - df_pair['static_y']
df_pair['pose_yaw'] = [get_relative_yaw(row['static_yaw'], row['dynamic_yaw']) for idx, row in df_pair.iterrows()]

In [22]:
df_pair.shape

(9951, 13)

In [23]:
df_pair.head()

,dynamic_idx,static_idx,distance,angle,static_x,static_y,static_yaw,dynamic_x,dynamic_y,dynamic_yaw,pose_x,pose_y,pose_yaw
0,0,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756,-0.183243,0.256744,-0.013123
1,1,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756,-0.183243,0.256744,-0.013123
2,2,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756,-0.183243,0.256744,-0.013123
3,3,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756,-0.183243,0.256744,-0.013123
4,4,3813,0.315429,0.013123,191.503235,129.493256,-179.987122,191.319992,129.75,179.999756,-0.183243,0.256744,-0.013123


In [24]:
print("Check how many corresponding yaw measurements are wrong:")
# TODO check for distance 
# TODO check if it is indeed the min
np.sum(df_pair['angle'] != np.abs(df_pair['pose_yaw']))

Check how many corresponding yaw measurements are wrong:


0

Save

In [25]:
df_pair.columns

Index(['dynamic_idx', 'static_idx', 'distance', 'angle', 'static_x',
       'static_y', 'static_yaw', 'dynamic_x', 'dynamic_y', 'dynamic_yaw',
       'pose_x', 'pose_y', 'pose_yaw'],
      dtype='object')

In [26]:
static_path_list = []
dynamic_path_list = []
for idx, row in df_pair.iterrows():
    static_path_list.append(df_static.iloc[int(row['static_idx'])]['path'])
    dynamic_path_list.append(df_dynamic.iloc[int(row['dynamic_idx'])]['path'])
    
df_pair['static_path'] = static_path_list
df_pair['dynamic_path'] = dynamic_path_list

In [27]:
fname = os.path.join(BASE_PATH, PAIR_FILE)

df_pair[['static_path', 'dynamic_path', 'pose_x', 'pose_y', 'pose_yaw']].to_csv(fname, index=False)

In [28]:
df_pair['dynamic_path'].values

array(['/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/dynamic/1/_out/1.ply',
       '/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/dynamic/1/_out/2.ply',
       '/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/dynamic/1/_out/3.ply',
       ...,
       '/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/dynamic/6/_out/6290.ply',
       '/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/dynamic/6/_out/6291.ply',
       '/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/dynamic/6/_out/6292.ply'],
      dtype=object)

In [29]:
fname

'/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_with_pose.csv'

Visualize any random pair for verification

In [30]:
import open3d as o3d
import copy
import transforms3d
o3d.__version__

'0.9.0.0'

In [31]:
def pose2matrix(translation_list, rotation_angle_list):
    trans_vec = np.array(translation_list)
    rot_ang = [np.deg2rad(ang) for ang in rotation_angle_list ]
    rot_mat = transforms3d.euler.euler2mat(rot_ang[0], rot_ang[1], rot_ang[2])
    zoom = np.ones(3)
    transform_mat = transforms3d.affines.compose(trans_vec, rot_mat, zoom)
    return transform_mat

def draw_registration_result(src_pcd, dst_pcd, x_pt, y_pt, theta):    
    src_pcd_tmp = copy.deepcopy(src_pcd)
    dst_pcd_tmp = copy.deepcopy(dst_pcd)
    
    src_pcd_tmp.paint_uniform_color([1, 0, 0])  # red source
    dst_pcd_tmp.paint_uniform_color([0, 0, 1])  # blue target
    
    transform_mat = pose2matrix([x_pt, y_pt, 0], [0,0,theta])
    dst_pcd_tmp.transform(transform_mat)
    
    visualizer = o3d.JVisualizer()
    visualizer.add_geometry(src_pcd_tmp)
    visualizer.add_geometry(dst_pcd_tmp)
    visualizer.show()

In [33]:
some_row = df_pair.iloc[np.random.choice(df_pair.shape[0])]
# some_row = df_pair.iloc[2]

print("Distance (in m): {} \nRelative Angle (in deg): {}".format(some_row['distance'], some_row['angle']))

some_static_pcd_path = df_static.iloc[int(some_row['static_idx'])]['path']
some_static_pcd = o3d.io.read_point_cloud(some_static_pcd_path)

some_dynamic_pcd_path = df_dynamic.iloc[int(some_row['dynamic_idx'])]['path']
some_dynamic_pcd = o3d.io.read_point_cloud(some_dynamic_pcd_path)

draw_registration_result(some_static_pcd, some_dynamic_pcd, 0, 0, 0)
# Static is red
# dynamic is blue

Distance (in m): 0.3676802231544385 
Relative Angle (in deg): 0.014678955543786287


JVisualizer with 2 geometries